# Clash of the Soccerbots

In this notebook we will control two jetbots to play each other 

### Loading the pretrained engine 

In [1]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Internally, the ``ObjectDetector`` class uses the TensorRT Python API to execute the engine that we provide.  It also takes care of preprocessing the input to the neural network, as
well as parsing the detected objects.  Right now it will only work for engines created using the ``jetbot.ssd_tensorrt`` package. That package has the utilities for converting
the model from the TensorFlow object detection API to an optimized TensorRT engine.

Next, let's initialize our camera.  Our detector takes 300x300 pixel input, so we'll set this when creating the camera.

> Internally, the Camera class uses GStreamer to take advantage of Jetson Nano's Image Signal Processor (ISP).  This is super fast and offloads
> a lot of the resizing computation from the CPU. 

In [2]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)

Now, let's execute our network using some camera input.  By default the ``ObjectDetector`` class expects ``bgr8`` format that the camera produces.  However,
you could override the default pre-processing function if your input is in a different format.

In [86]:
import traitlets
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg
import cv2

image=camera.value

image_widget = widgets.Image(format='jpeg', width=300, height=300)
width = int(image_widget.width)
height = int(image_widget.height)
display(image_widget)
detections = model(image)

for det in detections[0]:
    bbox = det['bbox']
    cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
    center = detection_center(det)
    print(center)
    print(float(0.8 + 0.4 * center[0]))
    print(float(0.8 - 0.4 * center[0]))
    print(norm(bbox))
          

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)
    
image_widget.value = bgr8_to_jpeg(image)

print(detections)

Image(value=b'', format='jpeg', height='300', width='300')

(-0.15496507287025452, -0.2294325903058052)
0.7380139708518982
0.8619860291481019
0.35306932589401774
[[{'label': 37, 'confidence': 0.554470419883728, 'bbox': [0.29458367824554443, 0.19462375342845917, 0.39548617601394653, 0.3465110659599304]}]]


If there are any COCO objects in the camera's field of view, they should now be stored in the ``detections`` variable.

### Control robot to follow central object

Now we want our robot to follow an object of the specified class.  To do this we'll do the following

1.  Detect objects matching the specified class
2.  Select object closest to center of camera's field of vision, this is the 'target' object
3.  Steer robot towards target object, otherwise wander
4.  If we're blocked by an obstacle, turn left


In [6]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

collision_model = torchvision.models.alexnet(pretrained=False)
collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 5)
collision_model.load_state_dict(torch.load('best_model.pth'))
device = torch.device('cuda')
collision_model = collision_model.to(device)

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Great, now let's initialize our robot so we can control the motors.

In [7]:
from jetbot import Robot

robot = Robot()

Finally, let's display all the control widgets and connect the network execution function to the camera updates.

In [95]:
from jetbot import bgr8_to_jpeg
import time

turn_left = 1
detstatus = 0
lookcount = 0
blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
red_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='red')
blue_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blue')
green_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='green')
yellow_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='yellow')
free_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='free')

image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
speed_widget = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')

display(widgets.VBox([
    widgets.HBox([image_widget, blocked_widget]),
    widgets.HBox([red_widget,blue_widget]),
    widgets.HBox([green_widget,yellow_widget]),
    free_widget,
    label_widget,
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.1
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.1
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection
        
def execute(change):
    global turn_left,detstatus,lookcount
    image = change['new']
    
    # execute collision model to determine if blocked
    collision_output = collision_model(preprocess(image)).detach().cpu()
    y = F.softmax(collision_output.flatten(), dim=0)
    y = y.flatten()
#     prob_blocked = float(F.softmax(collision_output.flatten(), dim=0)[0])
    prob_blocked = float(y[0])
    prob_free = float(y[1])
    prob_hit = float(y[2])
    prob_hitl = float(y[3])
    prob_hitr = float(y[4])
#     prob_red = float(y[5])
#     prob_yellow = float(y[6])
    blocked_widget.value = prob_blocked
    red_widget.value = prob_hitr
    blue_widget.value = prob_hitl
    green_widget.value = prob_hit
#     yellow_widget.value = prob_yellow
    free_widget.value = prob_free
    
    image_widget.value = bgr8_to_jpeg(image)
    
    # turn left if blocked
    if prob_free > 0.4 and detstatus == 0:
        if lookcount < 10:
            robot.left(0.2)
            time.sleep(0.0025)
            lookcount = lookcount + 1
        else:
            robot.forward(0.3)
            lookcount = 0
#         return
    elif prob_free > 0.4 and detstatus == 1:
        robot.forward(0.2)
        return
    elif prob_blocked > 0.5:
#         robot.backward(0.1)
#         time.sleep(0.25)
        robot.left(0.2)
        return
    elif prob_hit > 0.5:
        print("Entered into hit")
        camera.unobserve_all()
        time.sleep(1.0)
        robot.stop()
#         robot.backward(0.2)
        time.sleep(0.5)
        robot.forward(0.4)
        time.sleep(0.5)
        robot.stop()
        time.sleep(2)
        camera.observe(execute, names='value')
        print("Exiting hit")
        return
        
    # compute all detected objects
    detections = model(image)
    
    # draw all detections on image
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
    
    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == 37 or d['label'] == 53 or d['label']== 55 ]
    
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
    
    
        
    # otherwise go forward if no target detected
    if det is None:
#         robot.left(0.2)
        detstatus = 0
        robot.forward(0.2)
        
    # otherwsie steer towards target
    else:
        print("Entering ball detect")
        detstatus = 1
        # move robot forward and steer proportional target's x-distance from center
#         time.sleep(0.5)
        center = detection_center(det)
#         closeval = norm(det)
        print(center)
#         turn_g = turn_gain_widget.value
        turn_g = 0.4
        speed_g = 0.8
        robot.set_motors(
            float(speed_g + turn_g * center[0]),
            float(speed_g - turn_g * center[0])
        )
        time.sleep(1)
#         robot.set_motors(0.5,0.5)
#         if closeval < 0.4 and closeval > 0.1:
#             time.sleep(0.5)
#         elif closeval < 0.45 and closeval > 0.40:
#             time.sleep(0.7)
#         elif closeval > 0.45:
#             time.sleep(1)
    
        camera.unobserve_all()
        robot.stop()
        
        camera.observe(execute, names='value')
        print("Exiting ball detect")
    
    # update image widget
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})

Call the block below to connect the execute function to each camera frame update.

In [96]:
camera.unobserve_all()
camera.observe(execute, names='value')

Awesome!  If the robot is not blocked you should see boxes drawn around the detected objects in blue.  The target object (which the robot follows) will be displayed in green.

The robot should steer towards the target when it is detected.  If it is blocked by an object it will simply turn left.

You can call the code block below to manually disconnect the processing from the camera and stop the robot.

In [97]:


camera.unobserve_all()
time.sleep(1.0)
robot.stop()